# Scenarios

Multi-year planning with uncertain demand scenarios.

This notebook introduces:

- **Periods**: Multiple planning years with different conditions
- **Scenarios**: Uncertain futures (mild vs. harsh winter)
- **Scenario weights**: Probability-weighted optimization
- **Multi-dimensional data**: Parameters that vary by time, period, and scenario

## Setup

In [1]:
import pandas as pd
import xarray as xr

import flixopt as fx

fx.CONFIG.notebook()

flixopt.config.CONFIG

## The Planning Problem

We're designing a heating system with:

- **3 periods** (years): 2024, 2025, 2026 - gas prices expected to rise
- **2 scenarios**: "Mild Winter" (60% probability) and "Harsh Winter" (40% probability)
- **Investment decision**: Size of CHP unit (made once, works across all futures)

The optimizer finds the investment that minimizes **expected cost** across all scenarios.

## Define Dimensions

In [2]:
from data.tutorial_data import get_scenarios_data

data = get_scenarios_data()
timesteps = data['timesteps']
periods = data['periods']
scenarios = data['scenarios']
scenario_weights = data['scenario_weights']
heat_demand = data['heat_demand']
gas_prices = data['gas_prices']
elec_prices = data['elec_prices']

## Scenario-Dependent Demand Profiles

Heat demand differs significantly between mild and harsh winters:

In [3]:
# Visualize demand scenarios with plotly
demand_ds = xr.Dataset(
    {
        scenario: xr.DataArray(
            heat_demand[scenario].values,
            dims=['time'],
            coords={'time': timesteps},
        )
        for scenario in scenarios
    }
)
demand_ds.plotly.line(x='time', title='Heat Demand by Scenario')

## Build the Flow System

Initialize with all dimensions:

In [4]:
flow_system = fx.FlowSystem(
    timesteps=timesteps,
    periods=periods,
    scenarios=scenarios,
    scenario_weights=scenario_weights,
    name='Both Scenarios',
)
flow_system.add_carriers(
    fx.Carrier('gas', '#3498db', 'kW'),
    fx.Carrier('electricity', '#f1c40f', 'kW'),
    fx.Carrier('heat', '#e74c3c', 'kW'),
)

flow_system

FlowSystem
Timesteps: 168 (Hour) [2024-01-15 to 2024-01-21]
Periods: 3 (2024, 2025, 2026)
Scenarios: 2 (Mild Winter, Harsh Winter)
Status: ⚠


## Add Components

In [5]:
flow_system.add_elements(
    # === Buses ===
    fx.Bus('Electricity', carrier='electricity'),
    fx.Bus('Heat', carrier='heat'),
    fx.Bus('Gas', carrier='gas'),
    # === Effects ===
    fx.Effect('costs', '€', 'Total Costs', is_standard=True, is_objective=True),
    # === Gas Supply (price varies by period) ===
    fx.Source(
        'GasGrid',
        outputs=[
            fx.Flow(
                'Gas',
                bus='Gas',
                size=1000,
                effects_per_flow_hour=gas_prices,  # Array = varies by period
            )
        ],
    ),
    # === CHP Unit (investment decision) ===
    fx.linear_converters.CHP(
        'CHP',
        electrical_efficiency=0.35,
        thermal_efficiency=0.50,
        electrical_flow=fx.Flow(
            'P_el',
            bus='Electricity',
            # Investment optimization: find optimal CHP size
            size=fx.InvestParameters(
                minimum_size=0,
                maximum_size=100,
                effects_of_investment_per_size={'costs': 15},  # 15 €/kW/week annualized
            ),
        ),
        thermal_flow=fx.Flow('Q_th', bus='Heat'),
        fuel_flow=fx.Flow('Q_fuel', bus='Gas'),
    ),
    # === Gas Boiler (existing backup) ===
    fx.linear_converters.Boiler(
        'Boiler',
        thermal_efficiency=0.90,
        thermal_flow=fx.Flow('Q_th', bus='Heat', size=500),
        fuel_flow=fx.Flow('Q_fuel', bus='Gas'),
    ),
    # === Electricity Sales (revenue varies by period) ===
    fx.Sink(
        'ElecSales',
        inputs=[
            fx.Flow(
                'P_el',
                bus='Electricity',
                size=100,
                effects_per_flow_hour=-elec_prices,  # Negative = revenue
            )
        ],
    ),
    # === Heat Demand (varies by scenario) ===
    fx.Sink(
        'HeatDemand',
        inputs=[
            fx.Flow(
                'Q_th',
                bus='Heat',
                size=1,
                fixed_relative_profile=heat_demand,  # DataFrame with scenario columns
            )
        ],
    ),
)

## Run Optimization

In [6]:
flow_system.optimize(fx.solvers.HighsSolver(mip_gap=0.01));

## Analyze Results

### Optimal Investment Decision

In [7]:
chp_size = flow_system.statistics.sizes['CHP(P_el)']

pd.DataFrame(
    {
        'CHP Electrical [kW]': float(chp_size.max()),
        'CHP Thermal [kW]': float(chp_size.max()) * 0.50 / 0.35,
        'Expected Cost [EUR]': float(flow_system.solution['costs'].sum()),
    },
    index=['Optimal'],
).T

,Optimal
CHP Electrical [kW],99.671000
CHP Thermal [kW],142.387143
Expected Cost [EUR],77.793625


### Heat Balance by Scenario

See how the system operates differently in each scenario:

In [8]:
flow_system.statistics.plot.balance('Heat')

PlotResult(data=<xarray.Dataset> Size: 26kB
Dimensions:           (time: 169, period: 3, scenario: 2)
Coordinates:
  * time              (time) datetime64[ns] 1kB 2024-01-15 ... 2024-01-22
  * period            (period) int64 24B 2024 2025 2026
  * scenario          (scenario) object 16B 'Mild Winter' 'Harsh Winter'
Data variables:
    CHP(Q_th)         (time, period, scenario) float64 8kB -84.97 -131.7 ... nan
    Boiler(Q_th)      (time, period, scenario) float64 8kB -0.0 -5.787 ... nan
    HeatDemand(Q_th)  (time, scenario, period) float64 8kB 84.97 84.97 ... nan, figure=Figure({
    'data': [{'fillcolor': '#EF553B',
              'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f} kW<extra></extra>',
              'legendgroup': 'CHP(Q_th)',
              'line': {'color': '#EF553B', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'CHP(Q_th)',
              'orientation': 'v',
              'showlegend': True,
              'stackgroup': 'negative',
              'type': 'scatter',
              'x': array(['2024-01-15T00:00:00.000000000', '2024-01-15T01:00:00.000000000',
                          '2024-01-15T02:00:00.000000000', '2024-01-15T03:00:00.000000000',
                          '2024-01-15T04:00:00.000000000', '2024-01-15T05:00:00.000000000',
                          '2024-01-15T06:00:00.000000000', '2024-01-15T07:00:00.000000000',
                          '2024-01-15T08:00:00.000000000', '2024-01-15T09:00:00.000000000',
                          '2024-01-15T10:00:00.000000000', '2024-01-15T11:00:00.000000000',
                          '2024-01-15T12:00:00.000000000', '2024-01-15T13:00:00.000000000',
                          '2024-01-15T14:00:00.000000000', '2024-01-15T15:00:00.000000000',
                          '2024-01-15T16:00:00.000000000', '2024-01-15T17:00:00.000000000',
                          '2024-01-15T18:00:00.000000000', '2024-01-15T19:00:00.000000000',
                          '2024-01-15T20:00:00.000000000', '2024-01-15T21:00:00.000000000',
                          '2024-01-15T22:00:00.000000000', '2024-01-15T23:00:00.000000000',
                          '2024-01-16T00:00:00.000000000', '2024-01-16T01:00:00.000000000',
                          '2024-01-16T02:00:00.000000000', '2024-01-16T03:00:00.000000000',
                          '2024-01-16T04:00:00.000000000', '2024-01-16T05:00:00.000000000',
                          '2024-01-16T06:00:00.000000000', '2024-01-16T07:00:00.000000000',
                          '2024-01-16T08:00:00.000000000', '2024-01-16T09:00:00.000000000',
                          '2024-01-16T10:00:00.000000000', '2024-01-16T11:00:00.000000000',
                          '2024-01-16T12:00:00.000000000', '2024-01-16T13:00:00.000000000',
                          '2024-01-16T14:00:00.000000000', '2024-01-16T15:00:00.000000000',
                          '2024-01-16T16:00:00.000000000', '2024-01-16T17:00:00.000000000',
                          '2024-01-16T18:00:00.000000000', '2024-01-16T19:00:00.000000000',
                          '2024-01-16T20:00:00.000000000', '2024-01-16T21:00:00.000000000',
                          '2024-01-16T22:00:00.000000000', '2024-01-16T23:00:00.000000000',
                          '2024-01-17T00:00:00.000000000', '2024-01-17T01:00:00.000000000',
                          '2024-01-17T02:00:00.000000000', '2024-01-17T03:00:00.000000000',
                          '2024-01-17T04:00:00.000000000', '2024-01-17T05:00:00.000000000',
                          '2024-01-17T06:00:00.000000000', '2024-01-17T07:00:00.000000000',
                          '2024-01-17T08:00:00.000000000', '2024-01-17T09:00:00.000000000',
                          '2024-01-17T10:00:00.000000000', '2024-01-17T11:00:00.000000000',
                          '2024-01-17T12:00:00.000000000', '2024-01-17T13:00:00.000000000',
                        

### CHP Operation Patterns

In [9]:
flow_system.statistics.plot.heatmap('CHP(Q_th)')

PlotResult(data=<xarray.Dataset> Size: 10kB
Dimensions:    (timeframe: 8, timestep: 24, period: 3, scenario: 2)
Coordinates:
  * timeframe  (timeframe) object 64B '2024-01-15' '2024-01-16' ... '2024-01-22'
  * timestep   (timestep) object 192B '00:00' '01:00' ... '22:00' '23:00'
  * period     (period) int64 24B 2024 2025 2026
  * scenario   (scenario) object 16B 'Mild Winter' 'Harsh Winter'
Data variables:
    value      (timestep, timeframe, period, scenario) float64 9kB 84.97 ... nan, figure=Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'timeframe: %{x}<br>timestep: %{y}<br>CHP(Q_th)|flow_rate: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': array(['2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19',
                          '2024-01-20', '2024-01-21', '2024-01-22'], dtype=object),
              'xaxis': 'x',
              'y': array(['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00',
                          '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00',
                          '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'],
                         dtype=object),
              'yaxis': 'y',
              'z': {'bdata': ('5ZuWpeU9VUB1JJxVmKNSQI8Cqmvq21' ... 'NQaSp2VEDXc/GP8r1YQAAAAAAAAPh/'),
                    'dtype': 'f8',
                    'shape': '24, 8'}},
             {'coloraxis': 'coloraxis',
              'hovertemplate': 'timeframe: %{x}<br>timestep: %{y}<br>CHP(Q_th)|flow_rate: %{z}<extra></extra>',
              'name': '1',
              'type': 'heatmap',
              'x': array(['2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19',
                          '2024-01-20', '2024-01-21', '2024-01-22'], dtype=object),
              'xaxis': 'x2',
              'y': array(['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00',
                          '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00',
                          '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'],
                         dtype=object),
              'yaxis': 'y2',
              'z': {'bdata': ('5ZuWpeU9VUB1JJxVmKNSQI8Cqmvq21' ... 'NQaSp2VEDXc/GP8r1YQAAAAAAAAPh/'),
                    'dtype': 'f8',
                    'shape': '24, 8'}},
             {'coloraxis': 'coloraxis',
              'hovertemplate': 'timeframe: %{x}<br>timestep: %{y}<br>CHP(Q_th)|flow_rate: %{z}<extra></extra>',
              'name': '2',
              'type': 'heatmap',
              'x': array(['2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19',
                          '2024-01-20', '2024-01-21', '2024-01-22'], dtype=object),
              'xaxis': 'x3',
              'y': array(['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00',
                          '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00',
                          '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'],
                         dtype=object),
              'yaxis': 'y3',
              'z': {'bdata': ('5ZuWpeU9VUB1JJxVmKNSQI8Cqmvq21' ... 'NQaSp2VEDXc/GP8r1YQAAAAAAAAPh/'),
                    'dtype': 'f8',
                    'shape': '24, 8'}}],
    'frames': [{'data': [{'coloraxis': 'coloraxis',
                          'name': '0',
                          'type': 'heatmap',
                          'x': array(['2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19',
                                      '2024-01-20', '2024-01-21', '2024-01-22'], dtype=object),
                          'y': array(['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00',
                                      '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00',
                                      '16:00',

### Multi-Dimensional Data Access

Results include all dimensions (time, period, scenario):

In [10]:
flow_rates = flow_system.statistics.flow_rates

# Plot flow rates
flow_system.statistics.plot.flows()

PlotResult(data=<xarray.Dataset> Size: 66kB
Dimensions:           (period: 3, scenario: 2, time: 169)
Coordinates:
  * period            (period) int64 24B 2024 2025 2026
  * scenario          (scenario) object 16B 'Mild Winter' 'Harsh Winter'
  * time              (time) datetime64[ns] 1kB 2024-01-15 ... 2024-01-22
Data variables:
    GasGrid(Gas)      (time, period, scenario) float64 8kB 169.9 269.8 ... nan
    CHP(Q_fuel)       (time, period, scenario) float64 8kB 169.9 263.3 ... nan
    CHP(Q_th)         (time, period, scenario) float64 8kB 84.97 131.7 ... nan
    CHP(P_el)         (time, period, scenario) float64 8kB 59.48 92.16 ... nan
    Boiler(Q_fuel)    (time, period, scenario) float64 8kB 0.0 6.43 ... nan nan
    Boiler(Q_th)      (time, period, scenario) float64 8kB 0.0 5.787 ... nan nan
    ElecSales(P_el)   (time, period, scenario) float64 8kB 59.48 92.16 ... nan
    HeatDemand(Q_th)  (time, scenario, period) float64 8kB 84.97 84.97 ... nan, figure=Figure({
    'data': [{'hovertemplate': ('variable=GasGrid(Gas)<br>perio' ... '}<br>value=%{y}<extra></extra>'),
              'legendgroup': 'GasGrid(Gas), 2024',
              'line': {'color': '#636EFA', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'GasGrid(Gas), 2024',
              'showlegend': True,
              'type': 'scattergl',
              'x': array(['2024-01-15T00:00:00.000000000', '2024-01-15T01:00:00.000000000',
                          '2024-01-15T02:00:00.000000000', '2024-01-15T03:00:00.000000000',
                          '2024-01-15T04:00:00.000000000', '2024-01-15T05:00:00.000000000',
                          '2024-01-15T06:00:00.000000000', '2024-01-15T07:00:00.000000000',
                          '2024-01-15T08:00:00.000000000', '2024-01-15T09:00:00.000000000',
                          '2024-01-15T10:00:00.000000000', '2024-01-15T11:00:00.000000000',
                          '2024-01-15T12:00:00.000000000', '2024-01-15T13:00:00.000000000',
                          '2024-01-15T14:00:00.000000000', '2024-01-15T15:00:00.000000000',
                          '2024-01-15T16:00:00.000000000', '2024-01-15T17:00:00.000000000',
                          '2024-01-15T18:00:00.000000000', '2024-01-15T19:00:00.000000000',
                          '2024-01-15T20:00:00.000000000', '2024-01-15T21:00:00.000000000',
                          '2024-01-15T22:00:00.000000000', '2024-01-15T23:00:00.000000000',
                          '2024-01-16T00:00:00.000000000', '2024-01-16T01:00:00.000000000',
                          '2024-01-16T02:00:00.000000000', '2024-01-16T03:00:00.000000000',
                          '2024-01-16T04:00:00.000000000', '2024-01-16T05:00:00.000000000',
                          '2024-01-16T06:00:00.000000000', '2024-01-16T07:00:00.000000000',
                          '2024-01-16T08:00:00.000000000', '2024-01-16T09:00:00.000000000',
                          '2024-01-16T10:00:00.000000000', '2024-01-16T11:00:00.000000000',
                          '2024-01-16T12:00:00.000000000', '2024-01-16T13:00:00.000000000',
                          '2024-01-16T14:00:00.000000000', '2024-01-16T15:00:00.000000000',
                          '2024-01-16T16:00:00.000000000', '2024-01-16T17:00:00.000000000',
                          '2024-01-16T18:00:00.000000000', '2024-01-16T19:00:00.000000000',
                          '2024-01-16T20:00:00.000000000', '2024-01-16T21:00:00.000000000',
                          '2024-01-16T22:00:00.000000000', '2024-01-16T23:00:00.000000000',
                          '2024-01-17T00:00:00.000000000', '2024-01-17T01:00:00.000000000',
                          '2024-01-17T02:00:00.000000000', '2024-01-17T03:00:00.000000000',
                          '2024-01-17T04:00:00.000000000', '2024-01-17T05:00:00.000000000',
                          '2024-01-17T06:00:00.000000000', '2024-01-17T07:00:00.000000000',
                          '2024-

In [11]:
# CHP operation summary by scenario
chp_heat = flow_rates['CHP(Q_th)']

pd.DataFrame(
    {
        scenario: {
            'Avg [kW]': float(chp_heat.sel(scenario=scenario).mean()),
            'Max [kW]': float(chp_heat.sel(scenario=scenario).max()),
        }
        for scenario in scenarios
    }
)

,Mild Winter,Harsh Winter
Avg [kW],101.749508,133.619672
Max [kW],142.387143,142.387143


## Sensitivity: What if Only Mild Winter?

Compare optimal CHP size if we only planned for mild winters:

In [12]:
# Select only the mild winter scenario
fs_mild = flow_system.transform.sel(scenario='Mild Winter')
fs_mild.optimize(fx.solvers.HighsSolver(mip_gap=0.01))

chp_size_mild = float(fs_mild.statistics.sizes['CHP(P_el)'].max())
chp_size_both = float(chp_size.max())

pd.DataFrame(
    {
        'Mild Only': {'CHP Size [kW]': chp_size_mild},
        'Both Scenarios': {'CHP Size [kW]': chp_size_both},
        'Uncertainty Buffer': {'CHP Size [kW]': chp_size_both - chp_size_mild},
    }
)

,Mild Only,Both Scenarios,Uncertainty Buffer
CHP Size [kW],84.561091,99.671,15.109909


### Energy Flow Sankey

A Sankey diagram visualizes the total energy flows through the system:

In [13]:
flow_system.statistics.plot.sankey.flows()

PlotResult(data=<xarray.Dataset> Size: 2kB
Dimensions:  (link: 8)
Coordinates:
  * link     (link) int64 64B 0 1 2 3 4 5 6 7
    source   (link) <U11 352B 'Gas' 'Boiler' 'CHP' ... 'GasGrid' 'Heat'
    target   (link) <U11 352B 'Boiler' 'Heat' ... 'Gas' 'HeatDemand'
    label    (link) <U16 512B 'Boiler(Q_fuel)' ... 'HeatDemand(Q_th)'
    carrier  (link) <U11 352B 'gas' 'heat' 'electricity' ... 'gas' 'heat'
Data variables:
    value    (link) float64 64B 7.977e+03 7.179e+03 ... 1.234e+05 6.489e+04, figure=Figure({
    'data': [{'link': {'color': [rgba(52, 152, 219, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(241, 196, 15, 0.4), rgba(52, 152, 219, 0.4),
                                 rgba(231, 76, 60, 0.4), rgba(241, 196, 15, 0.4),
                                 rgba(52, 152, 219, 0.4), rgba(231, 76, 60, 0.4)],
                       'label': [Boiler(Q_fuel), Boiler(Q_th), CHP(P_el),
                                 CHP(Q_fuel), CHP(Q_th), ElecSales(P_el),
                                 GasGrid(Gas), HeatDemand(Q_th)],
                       'source': [1, 7, 4, 1, 4, 5, 2, 6],
                       'target': [7, 6, 5, 4, 6, 0, 1, 3],
                       'value': [7976.643544565088, 7178.979190108578,
                                 40394.744076395546, 115413.5545039872,
                                 57706.7772519936, 40394.744076395546,
                                 123390.19804855232, 64885.756442102196]},
              'node': {'color': [#AB63FA, #3498db, #636EFA, #FFA15A, #EF553B,
                                 #f1c40f, #e74c3c, #00CC96],
                       'label': [ElecSales, Gas, GasGrid, HeatDemand, CHP,
                                 Electricity, Heat, Boiler],
                       'line': {'color': 'black', 'width': 0.5},
                       'pad': 15,
                       'thickness': 20},
              'type': 'sankey'}],
    'layout': {'template': '...', 'title': {'text': 'Energy Flow'}}
}))

## Key Concepts

### Multi-Dimensional FlowSystem

```python
flow_system = fx.FlowSystem(
    timesteps=timesteps,      # Time dimension
    periods=periods,          # Planning periods (years)
    scenarios=scenarios,      # Uncertain futures
    scenario_weights=weights, # Probabilities
)
```

### Dimension-Varying Parameters

| Data Shape | Meaning |
|------------|----------|
| Scalar | Same for all time/period/scenario |
| Array (n_periods,) | Varies by period |
| Array (n_scenarios,) | Varies by scenario |
| DataFrame with columns | Columns match scenario names |
| Full array (time, period, scenario) | Full specification |

### Scenario Optimization

The optimizer minimizes **expected cost**:
$$\min \sum_s w_s \cdot \text{Cost}_s$$

where $w_s$ is the scenario weight (probability).

### Selection Methods

```python
# Select specific scenario
fs_mild = flow_system.transform.sel(scenario='Mild Winter')

# Select specific period
fs_2025 = flow_system.transform.sel(period=2025)

# Select time range
fs_day1 = flow_system.transform.sel(time=slice('2024-01-15', '2024-01-16'))
```

## Summary

You learned how to:

- Define **multiple periods** for multi-year planning
- Create **scenarios** for uncertain futures
- Use **scenario weights** for probability-weighted optimization
- Pass **dimension-varying parameters** (arrays and DataFrames)
- **Select** specific scenarios or periods for analysis

### Next Steps

- **[08a-Aggregation](08a-aggregation.ipynb)**: Speed up large problems with resampling and clustering
- **[08b-Rolling Horizon](08b-rolling-horizon.ipynb)**: Decompose large problems into sequential time segments